# Class or Sequence
* https://www.youtube.com/watch?v=NOu8jMZx3LY&list=PLjy4p-07OYzuy_lHcRW8lPTLPTTOmUpmi&index=16
* https://github.com/jeffheaton/app_deep_learning/blob/main/t81_558_class_03_5_pytorch_class_sequence.ipynb

In [6]:
try:
    import google.colab

    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False
    import torch

# Make use of a GPU or MPS (Apple) if one is available.  (see module 3.2)
import torch
has_mps = torch.backends.mps.is_built()
device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Note: using Google CoLab
Using device: cpu


## Defining Neural Networks as Sequences
In PyTorch, you represent a neural network as a sequence of layers. Each layer performs a specific computation on the input data and passes the transformed output to the next layer.

Example

In [ ]:
import torch
import torch.nn as nn

# Define the neural network architecture
model = nn.Sequential(
    nn.Linear(784, 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 10),
    nn.LogSoftmax(dim=1)
)

In this example, we created a neural network with three linear layers interspersed(点在する) with ReLU activation functions. The final layer utilizes the **LogSoftmax** activation, which we commonly use for classification tasks.

## Define Neural Networks as Classes
While the nn.Sequential approach is intuitive and convenient for many cases, it can become limiting when we need more flexibility in the network architecture. Defining neural networks as classes allows us to create custom architectures with complex behaviors and shareable components.


To define a neural network as a class, we typically subclass the **nn.Module** class provided by PyTorch. This base class offers essential functionality for organizing the network's parameters and handling computations during forward and backward passes.


Example

In [5]:
import torch
import torch.nn as nn

class CustomNetwork(nn.Module):
    def __init__(self):
        super(CustomNetwork, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 10)
        self.log_softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.log_softmax(x)
        return x

# Create an instance of the CustomNetwork
model = CustomNetwork()

In this example, we define a class called CustomNetwork that inherits from nn.Module. Inside the class, we describe the network's layers as attributes. The forward method specifies how the input flows through these layers during the forward pass.



By defining neural networks as classes, we can create more complex architectures, leverage conditional logic within the network, and encapsulate reusable components. This flexibility becomes particularly useful as we delve into advanced topics throughout the course.

## Choosing the Appropriate Method
While both the nn.Sequential and class-based approaches allow us to define neural networks; we will primarily utilize nn.Sequential in this course due to it simplicity and convenience. It offers an efficient way to construct networks with a linear sequence of layers.


However, understanding the class-based approach is crucial for deeper customization and building more intricate architectures. As you progress in mastering PyTorch, you'll encounter scenarios where defining networks as classes becomes necessary. You'll be well-equipped to tackle a wide range of neural network design challenges by grasping both methods.


In the upcoming chapters, we will explore various applications of nn.Sequential and learn advanced techniques to enhance the performance and capabilities of our neural networks.



Complex Example:

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from torch.autograd import Variable
from tqdm import tqdm

In [2]:
class Net(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Net, self).__init__()
        # Define each of the layers
        self.layer1 = nn.Linear(input_dim, 50)
        self.layer2 = nn.Linear(50, 25)
        self.layer3 = nn.Linear(25, output_dim)

    def forward(self, x):
        # Pass the input through each layer
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return x

In [7]:
# Load and preprocess data
df = pd.read_csv("https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", na_values=["NA", "?"])

# Replace missing horsepower values with median
df["horsepower"] = df["horsepower"].fillna(df["horsepower"].median())

# Convert pandas DataFrame to PyTorch tensors
x = torch.tensor(
    df[
        [
            "cylinders",
            "displacement",
            "horsepower",
            "weight",
            "acceleration",
            "year",
            "origin",
        ]
    ].values,
    device=device,
    dtype=torch.float32,
)
y = torch.tensor(df["mpg"].values, device=device, dtype=torch.float32) # regression


In [8]:
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [10]:
# Initialize the model, loss function, and optimizer
model = Net(input_dim=x.shape[1], output_dim=1).to(device)
model = torch.compile(model, backend="aot_eager").to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [11]:
# Training loop
for epoch in range(500):
    # Zero gradients
    optimizer.zero_grad()

    # Forward pass
    outputs = model(x).flatten()

    # Compute loss
    loss = loss_fn(outputs, y)

    # Backward pass and optimize
    loss.backward()
    optimizer.step()

    # Print loss every 10 epochs
    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch + 1}/{500}], Loss: {loss.item():.4f}")

Epoch [100/500], Loss: 142.9198
Epoch [200/500], Loss: 60.5489
Epoch [300/500], Loss: 35.1345
Epoch [400/500], Loss: 19.6866
Epoch [500/500], Loss: 13.5523
